<h1 style='text-align: center'>EDA for the Students Performance in Exams 📖</h1>

<p  style='text-align: center'>
This notebook is in <span style='color: green; font-weight: 700'>Active</span> state of development! Check out this notebook to see some updates as I update new stuff as oftern as I learn it!
<a style='font-weight:700' href='https://github.com/LilDataScientist'> My GitHub! </a></p>

<div style='text-align: center'>
    <img src='https://i.postimg.cc/rsQp1RXG/nguyen-dang-hoang-nhu-cb-Evo-Hb-Jn-IE-unsplash.jpg' width='600' />
</div>

<h1 style='background-color: #dae8fc; border: 1px solid #94add0; padding: 10px; font-weight: 400; text-align:center'>Research Questions</h1>

1. How effective is the test preparation course?
2. Which major factors contribute to test outcomes?
3. What would be the best way to improve student scores on each test?
4. Find out what are the most correlated features

<h1 style='background-color: #dae8fc; border: 1px solid #94add0; padding: 10px; font-weight: 400; text-align:center'>Table Of Contents</h1>

* Required Libraries
* Unique categorial values
* Features
* Missing values
* Quick Overview
* Quick Distribution Overview
* Compare people good at exact sciences with those who good at humanitarian sciences
* Correlation
    * Correlation between subjects
    * How lunch affects test score
    * Correlation between gender and study
    * Correlation between test score and preparation
    * Correlation between test score and 
* Feature Enginering
* Build model

<h1 style='background-color: #dae8fc; border: 1px solid #94add0; padding: 10px; font-weight: 400; text-align:center'>Required Libraries</h1>

In [ ]:
!pip install lazypredict
!pip install dataprep

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import lazypredict

from dataprep.eda import plot, plot_correlation, create_report, plot_missing
from sklearn.utils import shuffle
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from lazypredict.Supervised import LazyRegressor


sns.set_theme()

df = pd.read_csv('/kaggle/input/students-performance-in-exams/StudentsPerformance.csv')

df

<h1 style='background-color: #dae8fc; border: 1px solid #94add0; padding: 10px; font-weight: 400; text-align:center'>Unique categorial values</h1>

In [ ]:
categorical = df.select_dtypes(['category', 'object']).columns

for col in categorical:
    print('{} : {}'.format(col, ', '.join(df[col].unique())))

<h1 style='background-color: #dae8fc; border: 1px solid #94add0; padding: 10px; font-weight: 400; text-align:center'>Features</h1>

* Categorical  
    **gender** - Gender (female, male)  
    **race/ethnicity** - Ethnicity group (group A, group B, group C, group D, group E)  
    **parental level of education** - Parental education (master's degree, bachelor's degree, associate's degree, some college, high school)  
    **test preparation course** - Wheter person has studied for the course or not (none, completed)  
    
* Numerical  
    **math score** - Math test score   
    **reading score** - Reading test score  
    **writing score** - Writing test score  

<h1 style='background-color: #dae8fc; border: 1px solid #94add0; padding: 10px; font-weight: 400; text-align:center'>Missing values</h1>

In [ ]:
df.info()

#### Summary

We don't have missing values!

<h1 style='background-color: #dae8fc; border: 1px solid #94add0; padding: 10px; font-weight: 400; text-align:center'>Quick Overview</h1>

In [ ]:
plot(df)

<h1 style='background-color: #dae8fc; border: 1px solid #94add0; padding: 10px; font-weight: 400; text-align:center'>Quick Distribution Overview</h1>

In [ ]:
df.describe()

#### Summary

We can see that average person gets 66.09 for the math test, 68.05 for the writing test and 69.169 for the reading test.  

**Only math score has minimum score of 0! This could mean that students was not preparing good for exam! We will check this hypothes next!**

Also, math score has the lowest mean value! So, whether students really do not prepare for exam, or the math is hard it self!

<h1 style='background-color: #dae8fc; border: 1px solid #94add0; padding: 10px; font-weight: 400; text-align:center'>Compare people good at exact sciences with those who good at humanitarian sciences</h1>

In [ ]:
 # People with top 15% math score
top_15_math = df['math score'] > df['math score'].quantile(q=0.85)

top_15_math_math_mean = df[top_15_math]['math score'].mean()
top_15_math_writing_mean = df[top_15_math]['writing score'].mean()
top_15_math_reading_mean = df[top_15_math]['reading score'].mean()


# People with top 15% writing score
top_15_writing = df['writing score'] > df['writing score'].quantile(q=0.85)

top_15_writing_math_mean = df[top_15_writing]['math score'].mean()
top_15_writing_writing_mean = df[top_15_writing]['writing score'].mean()
top_15_writing_reading_mean = df[top_15_writing]['reading score'].mean()

# People with top 15% reading score
top_15_reading = df['reading score'] > df['reading score'].quantile(q=0.85) # People with top 15% reading score

top_15_reading_math_mean = df[top_15_reading]['math score'].mean()
top_15_reading_writing_mean = df[top_15_reading]['writing score'].mean()
top_15_reading_reading_mean = df[top_15_reading]['reading score'].mean()

# Summary DataFrame
compare_df = pd.DataFrame({
    'Math': [top_15_math_math_mean, top_15_writing_math_mean, top_15_reading_math_mean],
    'Writing': [top_15_math_writing_mean, top_15_writing_writing_mean, top_15_reading_writing_mean],
    'Rading': [top_15_math_reading_mean, top_15_writing_reading_mean, top_15_reading_reading_mean]
}, index=['Top 15% math score', 'Top 15% writing score', 'Top 15% reading score'])

compare_df

#### Summary

People who are very good at exact sciences are slitely worse than thosre who are good at humanitarian sciences. 
People who are very good at humanitarian sciences are slitely worse than those who are good at exact sciences. 

<h1 style='background-color: #defcdc; border: 1px solid #a1d194; padding: 10px; font-weight: 400; text-align:center'>Correlation</h1>

In [ ]:
full_df = pd.get_dummies(df)

f,ax = plt.subplots(figsize=(15, 15))

sns.heatmap(full_df.corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)
plt.show()

#### Summary & Our Hypothesis

We can see thay there is small negatve correlation between math, reading and writing score. This means that if person has not beed studied for the exam, then person has worse results. And vice versa, if person studied for the exam than results are better. But absolute value of correlation is not high, so we can not rely on this hypothesis.

Funny fact, that from heatmap we can see that person with standart lunch shows better results than the person with reduced free lunch, especialy in math test!

We clearly see that test score does not really rely on parental education.

Probably females study better than males

Perfoming good at reading makes you good at writing and vice versa. Since these subjects relates to humanitarian subjects

We can also see that people from different ethnicity group perfoms a little bit different.

<h1 style='background-color: #defcdc; border: 1px solid #a1d194; padding: 10px; font-weight: 400; text-align:center'>Correlation between subjects</h1>

In [ ]:
f,ax = plt.subplots(figsize=(10, 10))

sns.heatmap(df.corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)
plt.show()

#### Summary
Reading test score and writing test score has high correlation in comparison to the math test. This is pretty obisous, since they are quite similar, because it is both humanitarian subjects.

<h1 style='background-color: #defcdc; border: 1px solid #a1d194; padding: 10px; font-weight: 400; text-align:center'>How lunch affects test score</h1>

In [ ]:
f,ax = plt.subplots(figsize=(10, 10))

sns.heatmap(full_df[['math score', 'reading score', 'writing score', 'lunch_free/reduced', 'lunch_standard']].corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)
plt.show()

In [ ]:
df.groupby(['lunch']).mean()

In [ ]:
sns.relplot(
    data=full_df,
    x="math score", y="reading score", hue='writing score', col="lunch_standard");

#### Summary 

People who eat standard lunch perfom better on tests, especialy in math!     

Test score depends on food you eat! **So don't skip you lunch!**

<h1 style='background-color: #defcdc; border: 1px solid #a1d194; padding: 10px; font-weight: 400; text-align:center'>Correlation between gender and study</h1>

In [ ]:
f,ax = plt.subplots(figsize=(10, 10))

sns.heatmap(full_df[['math score', 'reading score', 'writing score', 'gender_male', 'gender_female']].corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)
plt.show()

In [ ]:
df.groupby(['gender']).mean()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(20,6))

sns.barplot(data=df, x="race/ethnicity", y="math score", hue='gender', palette="Blues_d", ax=axes[0]);
sns.barplot(data=df, x="race/ethnicity", y="reading score", hue='gender', palette="Blues_d", ax=axes[1]);
sns.barplot(data=df, x="race/ethnicity", y="writing score", hue='gender', palette="Blues_d", ax=axes[2]);

#### Summary 

Females are better at humanitarian sciences while males perfom better in exact sciences on average.

<h1 style='background-color: #defcdc; border: 1px solid #a1d194; padding: 10px; font-weight: 400; text-align:center'>Correlation between test score and preparation</h1>

In [ ]:
f,ax = plt.subplots(figsize=(10, 10))

sns.heatmap(full_df[['math score', 'reading score', 'writing score', 'test preparation course_none', 'test preparation course_completed']].corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)
plt.show()

In [ ]:
df.groupby(['test preparation course']).mean()

In [ ]:
sns.relplot(
    data=full_df,
    x="math score", y="reading score", hue='writing score', col="test preparation course_completed");

#### Summary 

Students who prepared for exam, perfom slitely better as we expected.

<h1 style='background-color: #defcdc; border: 1px solid #a1d194; padding: 10px; font-weight: 400; text-align:center'>Correlation between test score and ethnicity</h1>

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(20,6))

sns.barplot(data=df, x="race/ethnicity", y="math score", palette="Blues_d", ax=axes[0]);
sns.barplot(data=df, x="race/ethnicity", y="reading score", palette="Blues_d", ax=axes[1]);
sns.barplot(data=df, x="race/ethnicity", y="writing score", palette="Blues_d", ax=axes[2]);

In [ ]:
df.groupby(['race/ethnicity']).mean()

#### Summary

Group A perofoms the worst overall, while group E shows the best results over all subjects!

<h1 style='background-color: #fecfffcd; border: 1px solid #d194cf; padding: 10px; font-weight: 400; text-align:center'>Feature Engineering</h1>

<h1 style='background-color: #fecfffcd; border: 1px solid #d194cf; padding: 10px; font-weight: 400; text-align:center'>Build model</h1>

We will build a model to predict math test score!

Let's quickly find out what algorithm perfoms better!

In [ ]:
X = full_df.iloc[:, 3:]
y_math, y_reading, y_writing = full_df.iloc[:, 0], full_df.iloc[:, 1], full_df.iloc[:, 2]

X_train, X_test, y_train, y_test = train_test_split(X, y_math, test_size=0.25)

reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)
models, predictions = reg.fit(X_train, X_test, y_train, y_test)

models

#### Summary

Linear regression show small RMSE and it is not complicated algorithm, so we are not likely to overfit our dataset, we will choose Linear Rgression 

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

mean_squared_error(y_test, lr.predict(X_test))
mean_absolute_error(y_test, lr.predict(X_test))